```
Given two strings needle and haystack, return the index of the first occurrence of needle in haystack, or -1 if needle is not part of haystack.

 

Example 1:

Input: haystack = "sadbutsad", needle = "sad"
Output: 0
Explanation: "sad" occurs at index 0 and 6.
The first occurrence is at index 0, so we return 0.
Example 2:

Input: haystack = "leetcode", needle = "leeto"
Output: -1
Explanation: "leeto" did not occur in "leetcode", so we return -1.
 

Constraints:

1 <= haystack.length, needle.length <= 104
haystack and needle consist of only lowercase English characters.
```

In [1]:
const strStr =(haystack, needle) => {
  return haystack.indexOf(needle)
}

In [3]:
const haystack = 'sadbutsad'
const needle = 'sad'

console.log(strStr(haystack, needle))

0


In [2]:
const strStr = (haystack, needle) => {
  for(let i=0; i<haystack.length; i++) {
    if(haystack.slice(i, i+needle.length) === needle) {
      return i
    }
  }
  return -1
}


In [ ]:
// KMP
const strStrKMP = function (haystack, needle) {
  // needleが空文字列の場合、needleはhaystackの先頭に存在するとみなす
  if (needle.length === 0) {
    return 0
  }

  const n = haystack.length // haystackの長さ（この例では9）
  const m = needle.length // needleの長さ（この例では3）

  // needleの長さがhaystackの長さより大きい場合、needleはhaystackに含まれない
  if (m > n) {
    return -1
  }

  // needleの各位置における「最長接頭辞であり、かつ接尾辞でもある文字列の長さ」を計算する関数
  const computeLPSArray = (pattern) => {
    const lps = new Array(pattern.length).fill(0) // lps配列を0で初期化（この例では[0, 0, 0]になる）
    let length = 0 // 現在のマッチしている接頭辞の長さ
    lps[0] = 0 // lpsの最初の要素は常に0
    let i = 1 // lps配列の2番目の要素から計算を開始

    while (i < pattern.length) {
      // 現在の文字と、lengthの位置の文字が一致する場合
      if (pattern[i] === pattern[length]) {
        length++ // マッチした長さを増やす
        lps[i] = length // lps[i]に現在のマッチした長さを記録
        i++ // 次の文字へ
      } else {
        // 一致しない場合
        if (length !== 0) {
          // lengthが0でない場合、前の最長接頭辞の長さを参照する
          length = lps[length - 1]
          // ここでiはインクリメントしない（再度同じ位置で比較するため）
        } else {
          // lengthが0の場合、現在のlps[i]は0のまま
          lps[i] = 0
          i++ // 次の文字へ
        }
      }
    }
    return lps // 計算されたlps配列を返す
  }

  // needleのLPS配列を計算する（この例ではneedle = "sad"なので、lpsは[0, 0, 0]になる）
  const lps = computeLPSArray(needle)
  let i = 0 // haystackの現在位置を示すインデックス
  let j = 0 // needleの現在位置を示すインデックス

  // haystack全体を走査するループ
  while (i < n) {
    // needleのj番目の文字とhaystackのi番目の文字が一致する場合
    if (needle[j] === haystack[i]) {
      j++ // needleの比較位置を次へ
      i++ // haystackの比較位置を次へ
    }

    // needleの最後まで一致した場合（needleが見つかった場合）
    if (j === m) {
      return i - j // haystack内でのneedleの開始位置を返す
    }
    // needleの文字とhaystackの文字が一致しない場合
    else if (i < n && needle[j] !== haystack[i]) {
      // jが0でない場合（部分的にマッチしていた場合）
      if (j !== 0) {
        // needleの比較位置を、直前のLPSの値に戻す
        j = lps[j - 1]
        // これにより、無駄な比較をスキップできる
      } else {
        // jが0の場合（needleの最初の文字から一致しなかった場合）
        i++ // haystackの比較位置を次へ
      }
    }
  }

  // ループが終了してもneedleが見つからなかった場合
  return -1
}


In [4]:
// rolling hash
const strStrRollingHash = function (haystack, needle) {
  // needleが空文字列の場合、needleはhaystackの先頭に存在するとみなす
  if (needle.length === 0) {
    return 0
  }

  const n = haystack.length // haystackの長さ（この例では9）
  const m = needle.length // needleの長さ（この例では3）

  // needleの長さがhaystackの長さより大きい場合、needleはhaystackに含まれない
  if (m > n) {
    return -1
  }

  const base = 26 // 小文字のアルファベットは26種類なので、基数として26を使用
  const modulus = 10 ** 9 + 7 // ハッシュ値が大きくなりすぎないようにするための剰余

  // 文字列のハッシュ値を計算する関数
  const calculateHash = (str) => {
    let hash = 0
    for (let i = 0; i < str.length; i++) {
      // 各文字を数値に変換（'a'を1、'b'を2、...）し、基数を掛けて足し合わせる
      hash =
        (hash * base + (str.charCodeAt(i) - 'a'.charCodeAt(0) + 1)) % modulus
    }
    return hash
  }

  // needleのハッシュ値を計算（"sad" のハッシュ値を計算）
  const needleHash = calculateHash(needle)

  // haystackの最初のneedleと同じ長さの部分文字列のハッシュ値を計算（"sad" のハッシュ値を計算）
  let haystackHash = calculateHash(haystack.substring(0, m))

  // 最初の部分文字列のハッシュ値がneedleのハッシュ値と一致し、かつ文字列自体も一致する場合
  if (needleHash === haystackHash && haystack.substring(0, m) === needle) {
    return 0 // インデックス0で見つかった
  }

  let power = 1
  // ローリングハッシュを効率的に計算するために、基数の (m-1) 乗を事前に計算
  for (let i = 0; i < m - 1; i++) {
    power = (power * base) % modulus
  }

  // haystackの残りの部分を走査するループ
  for (let i = m; i < n; i++) {
    // 先頭の文字のハッシュ値への影響を取り除く
    haystackHash =
      (haystackHash -
        (haystack.charCodeAt(i - m) - 'a'.charCodeAt(0) + 1) * power) %
      modulus
    if (haystackHash < 0) {
      haystackHash += modulus // 負の値になった場合はmodulusを足して正の値に戻す
    }
    // 新しい末尾の文字のハッシュ値への影響を追加
    haystackHash =
      (haystackHash * base + (haystack.charCodeAt(i) - 'a'.charCodeAt(0) + 1)) %
      modulus

    // 現在の部分文字列のハッシュ値がneedleのハッシュ値と一致し、かつ文字列自体も一致する場合
    if (
      needleHash === haystackHash &&
      haystack.substring(i - m + 1, i + 1) === needle
    ) {
      return i - m + 1 // 現在の部分文字列の開始インデックスを返す
    }
  }

  // ループが終了してもneedleが見つからなかった場合
  return -1
}